In [ ]:
import datetime
import os
import random

import tensorflow as tf
import numpy as np

from tensorflow.contrib.layers import batch_norm
from tensorflow.contrib.layers import variance_scaling_initializer
from tensorflow.contrib.layers import conv2d
from tensorflow.contrib.layers import flatten
from tensorflow.contrib.layers import max_pool2d
from tensorflow.contrib.layers import avg_pool2d
from tensorflow.contrib.layers import dropout
from tensorflow.contrib.layers import fully_connected

from PIL import ImageEnhance
from PIL import Image

# path to dataset
DATA_ROOT = r"/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENCE/KAGGLE_COMPETITIONS/Dogs_vs_Cats_Redux_Kernels_Edition"
assert (os.path.exists(DATA_ROOT))

TRAIN_DATA = os.path.join(DATA_ROOT, "train")
assert (os.path.exists(TRAIN_DATA))

TEST_DATA = os.path.join(DATA_ROOT, "test")
assert (os.path.exists(TEST_DATA))

BRIGHTNESS_FACTOR_MIN = 0.2
BRIGHTNESS_FACTOR_MAX = 1.8

CONTRAST_FACTOR_MIN = 0.2
CONTRAST_FACTOR_MAX = 1.8

SHARPENESS_FACTOR_MIN = 0
SHARPENESS_FACTOR_MAX = 2

SATURATION_FACTOR_MIN = 0
SATURATION_FACTOR_MAX = 2

def random_enhance_pil(
    image, brightness=True, contrast=True, saturation=True, sharpness=True):

    new_image = image

    if brightness:
        enhancer = ImageEnhance.Brightness(new_image)
        new_image = enhancer.enhance(
          random.uniform(BRIGHTNESS_FACTOR_MIN, BRIGHTNESS_FACTOR_MAX))
    
    if contrast:
        enhancer = ImageEnhance.Contrast(new_image)
        new_image = enhancer.enhance(
          random.uniform(CONTRAST_FACTOR_MIN, CONTRAST_FACTOR_MAX))

    if saturation:
        enhancer = ImageEnhance.Color(new_image)
        new_image = enhancer.enhance(
          random.uniform(SATURATION_FACTOR_MIN, SATURATION_FACTOR_MAX))

    if sharpness:
        enhancer = ImageEnhance.Sharpness(new_image)
        new_image = enhancer.enhance(
          random.uniform(SHARPENESS_FACTOR_MIN, SHARPENESS_FACTOR_MAX))

    return new_image


def resize_image_pil(
    image, size, random_enhance=True, pad=True, resample_filter=Image.LANCZOS):

  if random_enhance:
    image = random_enhance_pil(image)

  width, height = image.size

  if height > width:
    new_height = size
    ratio = new_height / height
    new_width = int(width * ratio)

  elif width > height:
    new_width = size
    ratio = new_width / width
    new_height = int(height * ratio)

  else:
    new_width = size
    new_height = size

  image_resize = image.resize((new_width, new_height), resample=resample_filter)

  if pad:
    image_pad = Image.new("RGB", (size, size), (0, 0, 0))
    ulc = ((size - new_width) // 2, (size - new_height) // 2)

    image_pad.paste(image_resize, ulc)

    return image_pad
  else:
    return image_resize


def get_label(file_name):
  label = file_name.split(".")[0]

  if label == "cat":
    return 0
  elif label == "dog":
    return 1

  # if label == "cat":
  # return [1.0, 0.0]
  # elif label == "dog":
  # return [0.0, 1.0]


def get_next_batch(iteration, batch_size, dataset):
  if batch_size * (iteration + 1) > len(dataset):
    return dataset[batch_size * iteration:]
  else:
    return dataset[batch_size * iteration:batch_size * (iteration + 1)]


def image_to_nparray_pil(image):
  return np.array(image.getdata()).reshape((SIZE, SIZE, 3))


def random_enhance_tf(image, brightness=True, contrast=True, saturation=True):
  new_image = image

  if brightness:
    seed = datetime.datetime.now().second
    new_image = tf.image.random_brightness(new_image, 5, seed=seed)

  if contrast:
    seed = datetime.datetime.now().second
    new_image = tf.image.random_contrast(new_image, 0.1, 1.5, seed=seed)

  if saturation:
    seed = datetime.datetime.now().second
    new_image = tf.image.random_saturation(new_image, 0., 1.5, seed=seed)

  return new_image


def random_flip_tf(image, vertical=True, horizontal=True):
  new_image = image

  if vertical:
    seed = datetime.datetime.now().second
    new_image = tf.image.random_flip_up_down(new_image, seed=seed)

  if horizontal:
    seed = datetime.datetime.now().second
    new_image = tf.image.random_flip_left_right(new_image, seed=seed)

  return new_image


def mse_error(inputs, labels):
  inputs_flatten = tf.contrib.layers.flatten(inputs)
  labels_flatten = tf.contrib.layers.flatten(labels)

  difference = tf.subtract(labels_flatten, inputs_flatten)

  return tf.square(difference)


In [ ]:
batch_size = 5

results = {}

with tf.Session() as sess:

    # we need to initialize all variables first
    init.run()
    
    saver.restore(sess, "../ml_engine/logs/checkpoint_499500.ckpt")
        
    DATASETS = sorted([os.path.join(TEST_DATA, i) for i in os.listdir(TEST_DATA)], 
                      key=lambda i: int(os.path.splitext(os.path.basename(i))[0]))

#     print([os.path.basename(i) for i in DATASETS])

    ids = [os.path.splitext(os.path.basename(i))[0] for i in DATASETS]

    next_batch = DATASETS

    x_batch = []

    for file in next_batch:
        image = Image.open(file)
        image = resize_image_pil(image, SIZE, random_enhance=False)
        x_batch.append(image)

    x_batch = [image_to_nparray_pil(image) for image in x_batch]

    predict_val, probability_val = sess.run(
        [predict, probability], 
        feed_dict={
            x: x_batch, 
            is_training: False,
        }
    )
    
    for index, image_id in enumerate(ids):
        results[image_id] = probability_val[index][1]

with open("classification.csv", "w", newline="") as f:
    writer = csv.writer(
        f, 
        delimiter=",",
        quotechar="|", 
        quoting=csv.QUOTE_MINIMAL)
    
    writer.writerow(["id", "label"])
    
    for key, value in results.items():
        writer.writerow([str(key), str(value)])
        
print("FINISH!!")